# Progetto Lar Generators

| Nome| Matricola | E-mail | Profilo Github |
|:---|:---|:---|:---|
|Paolo Mazzitti|502042|pao.mazzitti@stud.uniroma3.it| [https://github.com/paolomazzitti](https://github.com/paolomazzitti) |
| Matteo Colonnello|527289|mat.colonnello@stud.uniroma3.it|[https://github.com/MatteoColonnello](https://github.com/MatteoColonnello)|
| Martina Falanga|522705|mar.falanga@stud.uniroma3.it|[https://github.com/MartinaFalanga](https://github.com/MartinaFalanga) |

Per avviare una istanza Jupyter con 8 threads, eseguire il seguente codice dal proprio terminale e selezionare il kernel appena creato.

```julia
using IJulia
IJulia.installkernel("Julia 8 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "8",
))
````

In [1]:
using Base.Threads
Threads.nthreads()

8

## Codice funzionante versione originale
Per mostrare le differenze prestazionali tra la versione originale del progetto e quella ottimizzata dal gruppo è necessario eseguire la cella sottostante. Il codice, infatti, presentava inizialmente degli errori che non ne permettavano l'esecuzione.

In [2]:
function setTile(box)
    tiles = [[9,1,5],[8,0,4],[10,2,6]]
    b1,b2,b3,b4 = box
    function tileCode(point)
        x, y = point
        code = 0
        if y>b1 code=code|1 end
        if y<b2 code=code|2 end
        if x>b3 code=code|4 end
        if x<b4 code=code|8 end
        return code
    end
    return tileCode
end

function pointInPolygonClassification(V,EV)
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                if c_int == 0 return "p_on"
                elseif c_int == 4 count += 1 end
            elseif c_edge == 15
                x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
                if x_int > x count += 1
                elseif x_int == x return "p_on" end
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 count += 1
            elseif c_edge == 11 count = count
            elseif c_edge == 1
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(1,2,status,count) end
            elseif c_edge == 2
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(2,1,status,count) end
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(1,2,status,count) end
            elseif c_edge == 6
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(2,1,status,count) end
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
            return "p_in"
        else
            return "p_out"
        end
    end
    return pointInPolygonClassification0
end

function chainbasis2polygonsEx(V, copEV, copFE)
    FE = [findnz(copFE[k, :])[1] for k = 1:copFE.m]
    EV = [findnz(copEV[k, :])[1] for k = 1:copEV.m]

    FEs = []
    EVs = []
    FVs = []
    
    for f = 1:copFE.m
        push!(FEs, collect(Set(cat([e for e in FE[f]]; dims=1))))
        push!(EVs, [EV[e] for e in FE[f]])
        push!(FVs, collect(Set(cat([EV[e] for e in FE[f]]; dims=1))))
    end
    polygons = collect(zip(EVs, FVs, FEs))
    W = convert(Lar.Points, V')
    return W, polygons, FE
end


function internalpoints2dEx(W, copEV, copFE)
    U, pols, FE = chainbasis2polygonsEx(W, copEV, copFE)
    internalpoints = []
    for f = 1:length(pols)
        (EV, FV, FE) = pols[f]
        internalpoint = Lar.getinternalpoint2d(W, EV, FV, f, copEV, copFE)
        push!(internalpoints, internalpoint)
    end
    return internalpoints
end


function bool2dEx(assembly)
    
    V, EV = Lar.struct2lar(assembly)
    cop_EW = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
    W = convert(Lar.Points, V')
   
    W, copEV, copFE = Lar.Arrangement.planar_arrangement(W::Lar.Points, cop_EW::Lar.ChainOp)
    
    innerpoints = internalpoints2dEx(W, copEV, copFE[1:end, :])
    
    listOfModels = Lar.evalStruct(assembly)
    inputfacenumbers = [length(listOfModels[k][2]) for k = 1:length(listOfModels)]
    
    boolmatrix = BitArray(undef, length(innerpoints), length(listOfModels))
    containmenttest = Lar.testinternalpoint2d(listOfModels)
    for (k, point) in enumerate(innerpoints)
        cells = containmenttest(point)
        for l in cells
            boolmatrix[k, l] = 1
        end
    end
    return W, copEV, copFE, boolmatrix
end

bool2dEx (generic function with 1 method)

## Codice sviluppato e ottimizzato

In [3]:
"""
    setTileBool(box)(point)

# Explanation

Returns the tile code of an edge.

# Arguments

- `box::Tuple{Float64, Float64, Float64, Float64}`
- `point::Vector{Float64}`

# Return

- `tileCodeBool::Int64`
"""
@inline function setTileBool(box)
    b1, b2, b3, b4 = box
    function tileCodeBool(point)
        x, y = point
        code = 0
        if y > b1
            code = code | 1
        end
        if y < b2
            code = code | 2
        end
        if x > b3
            code = code | 4
        end
        if x < b4
            code = code | 8
        end
        return code
    end
    return tileCodeBool
end

setTileBool

In [4]:
"""
    pointInPolygonClassificationBool(V, EV)(pnt)

# Explanation

Determines whether a point is inside or outside a polygon. It is a specific version used for the bool2d function.

# Arguments
- `V::Adjoint{Float64, Matrix{Float64}}`
- `EV::Vector{Vector{Int64}}`
- `pnt::Vector{Float64}`

# Return
- `p_in::String` or `p_out::String`
"""
function pointInPolygonClassificationBool(V, EV)
    function pointInPolygonClassification0Bool(pnt)
        x, y = pnt
        xmin, xmax, ymin, ymax = x, x, y, y
        tilecodeBool = setTileBool([ymax, ymin, xmax, xmin])
        count = 0
        for edge in EV
            p1, p2 = V[:, edge[1]], V[:, edge[2]]
            c1, c2 = tilecodeBool(p1), tilecodeBool(p2)
            (x1, y1), (x2, y2) = p1, p2
            c_edge, c_int = c1 ⊻ c2, c1 & c2

            if c_edge == 3 && c_int == 4
                count += 1
            elseif c_edge == 15
                x_int = ((y - y2) * (x1 - x2) / (y1 - y2)) + x2
                if x_int > x
                    count += 1
                end
            end
        end
        if (round(count) % 2) == 1
            return "p_in"
        else
            return "p_out"
        end
    end
    return pointInPolygonClassification0Bool
end

pointInPolygonClassificationBool

In [5]:
"""
    settestpoints2d(W, f, copEV, copFE)

# Explanation

Given a polygon, it returns two points, one inside and one outside it.

# Arguments
- `W::Matrix{Float64}`
- `f::Int64`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`

# Return
- `(ptest1, ptest2)::Tuple{Vector{Float64}, Vector{Float64}}`
"""
@inline function settestpoints2d(W, f, copEV, copFE)
    e = findnz(copFE[f, :])[1][1]
    v1, v2 = findnz(copEV[e, :])[1]
    t = W[v2, :] - W[v1, :]
    n = [-t[2], t[1]]
    p0 = (W[v1, :] + W[v2, :]) ./ 2
    ϵ = 1.0e-4
    ptest1 = p0 + ϵ * n
    ptest2 = p0 - ϵ * n
    return ptest1, ptest2
end

settestpoints2d

In [6]:
"""
    getinternalpoint2d(W, f, copEV, copFE)

# Explanation

Returns an interior point in the polygon. 

# Arguments

- `W::Matrix{Float64}`
- `f::Int64`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`

# Return

- `ptest1::Vector{Float64}` or `ptest2::Vector{Float64}`
"""
function getinternalpoint2d(W, f, copEV, copFE)
    ptest1, ptest2 = settestpoints2d(W, f, copEV, copFE)
    edges = [findnz(copEV[e, :])[1] for e in findnz(copFE[f, :])[1]]
    V = W'
    classify = pointInPolygonClassificationBool(V, edges)
    if classify(ptest1) == "p_in"
        return ptest1
    else
        return ptest2
    end
end

getinternalpoint2d

In [7]:
"""
    internalpoints2d(W, copEV, copFE)

# Explanation

Returns, for each atom of the decomposition, an internal point.

# Arguments
- `W::Matrix{Float64}`
- `copEV::SparseMatrixCSC{Int8, Int64}`
- `copFE::SparseMatrixCSC{Int8, Int64}`

# Return
- `internalpoints::Vector{Vector{Float64}}`
"""
function internalpoints2d(W, copEV, copFE)
    internalpoints = Vector{Vector{Float64}}(undef, copFE.m)
    @threads for f = 1:copFE.m
        internalpoint = getinternalpoint2d(W, f, copEV, copFE)
        @inbounds internalpoints[f] = internalpoint
    end
    return internalpoints
end

internalpoints2d

In [8]:
"""
	testinternalpoint2d(listOfModels)(testpoint)

# Explanation

Given a point, it returns the list of models containing it.
    
# Arguments
- `listOfModels::Vector{Any}`
- `testpoint::Vector{Float64}`

# Return
- `intersectedFaces: Vector{Int64}`
"""
function testinternalpoint2d(listOfModels)
    function testinternalpoint0(testpoint)

        intersectedFaces = Vector{Int64}()

        for (k, model) in enumerate(listOfModels)
            verts, edges = model
            classify = pointInPolygonClassificationBool(verts, edges)
            inOut = classify(testpoint)
            if inOut == "p_in"
                push!(intersectedFaces, k)
            end
        end

        return intersectedFaces
    end
    return testinternalpoint0
end

testinternalpoint2d

In [9]:
"""
    bool2d(assembly)

# Arguments
- `assembly::LinearAlgebraicRepresentation.Struct`
    
# Return
- `W::Matrix{Float64}`
- `copEV::SparseMatrixCSC{Int8,Int64}`
- `copFE::SparseMatrixCSC{Int8,Int64}`
- `boolmatrix::BitMatrix`

# Usage
```julia
using SparseArrays, LARgenerators, Base.Threads, SparseArrays, IntervalTrees, LinearAlgebra
import LinearAlgebraicRepresentation as Lar

V, (VV, EV, FV) = Lar.cuboidGrid([1, 1], true)
square = V, EV

assembly = Lar.Struct([
    Lar.Struct([Lar.t(0,0), Lar.r(0), square])
    Lar.Struct([Lar.t(0,0.1), Lar.r(0.1), square])
    Lar.Struct([Lar.t(0,0.2), Lar.r(0.2), square])
    Lar.Struct([Lar.t(0,0.3), Lar.r(0.3), square])
    Lar.Struct([Lar.t(0,0.4), Lar.r(0.4), square])
])

W, copEV, copFE, boolmatrix = LARgenerators.bool2d(assembly)
```
"""
function bool2d(assembly)

    V::Matrix{Float64}, EV::Vector{Vector{Int64}} = Lar.struct2lar(assembly)
    cop_EW = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
    Z = convert(Lar.Points, V')
    W::Matrix{Float64}, copEV::SparseMatrixCSC{Int8,Int64}, copFE::SparseMatrixCSC{Int8,Int64} = Lar.planar_arrangement(Z::Lar.Points, cop_EW::Lar.ChainOp)
    innerpoints = internalpoints2d(W, copEV, copFE)
    listOfModels = Lar.evalStruct(assembly)
    boolmatrix = BitArray(undef, copFE.m, length(listOfModels))

    containmenttest = testinternalpoint2d(listOfModels)

    for (k, point) in collect(enumerate(innerpoints))
        cells = containmenttest(point)
        for l in cells
            @inbounds boolmatrix[k, l] = 1
        end
    end
    return W, copEV, copFE, boolmatrix
end

bool2d

## Esempio codice sviluppato ed ottimizzato
Esempio parametrico con $t$ quadrati.

In [10]:
using ViewerGL, SparseArrays, BenchmarkTools
import LinearAlgebraicRepresentation as Lar
GL = ViewerGL

V, (VV, EV, FV) = Lar.cuboidGrid([1, 1], true)
square = V, EV

assembly = Lar.Struct([
    Lar.Struct([Lar.t(0,0), Lar.r(0), square])
    Lar.Struct([Lar.t(0,0.1), Lar.r(0.1), square])
    Lar.Struct([Lar.t(0,0.2), Lar.r(0.2), square])
    Lar.Struct([Lar.t(0,0.3), Lar.r(0.3), square])
    Lar.Struct([Lar.t(0,0.4), Lar.r(0.4), square])
])

V, EV = Lar.struct2lar(assembly)
GL.VIEW([GL.GLGrid(V, EV, GL.COLORS[1], 1), GL.GLFrame2]);

W, copEV, copFE, boolmatrix = bool2d(assembly)

A = boolmatrix[:, 1]
B = boolmatrix[:, 2]
C = boolmatrix[:, 3]
D = boolmatrix[:, 4]
E = boolmatrix[:, 5]

AorB = A .| B .| C .| D .| E
AandB = A .& B .& C .& D .& E
AxorB = A .⊻ B .⊻ C .⊻ D .⊻ E

union = Matrix(copFE)' * Int.(AorB)
intersection = Matrix(copFE)' * Int.(AandB)
xor = Matrix(copFE)' * AxorB

V = convert(Lar.Points,W')
EV = Lar.cop2lar(copEV)
EVor = [ev for (k,ev) in enumerate(EV) if abs(union[k])==1 ]
EVand = [ev for (k,ev) in enumerate(EV) if abs(intersection[k])==1 ]
EVxor = [ev for (k,ev) in enumerate(EV) if abs(xor[k])==1 ]

GL.VIEW([ GL.GLGrid(V,EVor, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVand, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVxor, GL.COLORS[1],1), GL.GLFrame2 ]);

model = (V,[VV, EVor])
GL.VIEW( GL.numbering(.5)( model,GL.COLORS[1],0.1 ) );

model = (V,[VV, EVand])
GL.VIEW( GL.numbering(.5)( model,GL.COLORS[1],0.1 ) );

model = (V,[VV, EVxor])
GL.VIEW( GL.numbering(.5)( model,GL.COLORS[1],0.1 ) );

# Analisi delle prestazioni
Di seguito sono confrontati i tempi di esecuzioni delle funzioni ottimizzate con le loro versioni originali presenti sul notebook.

## Benchmark della funzione `bool2d` originale
I risultati dei benchmark, condotti su un MacBook Air (M1, 2020) 8 cores, sono commentati dentro ogni cella.

In [20]:
using BenchmarkTools
@btime bool2dEx($assembly)

# Risultato test: 6.540 ms (47573 allocations: 2.95 MiB)

  6.540 ms (47573 allocations: 2.95 MiB)


([0.0 0.0; 0.0 1.0; … ; 0.921061 0.7894183; 0.5316427 1.7104793], sparse([1, 7, 5, 8, 4, 5, 60, 71, 3, 4  …  58, 59, 69, 70, 63, 74, 64, 73, 70, 74], [1, 1, 2, 2, 3, 3, 3, 3, 4, 4  …  42, 42, 42, 42, 43, 43, 44, 44, 45, 45], Int8[-1, -1, -1, -1, -1, 1, -1, -1, -1, 1  …  1, 1, 1, -1, 1, -1, 1, 1, 1, 1], 74, 45), sparse([11, 1, 18, 2, 8, 3, 9, 4, 10, 11  …  16, 29, 30, 3, 4, 5, 27, 15, 28, 30], [1, 2, 2, 3, 3, 4, 4, 5, 5, 6  …  69, 69, 70, 71, 71, 72, 72, 73, 73, 74], Int8[-1, 1, -1, 1, -1, 1, -1, 1, -1, 1  …  1, -1, 1, -1, 1, -1, 1, -1, 1, -1], 30, 74), Bool[1 1 … 0 1; 1 1 … 0 1; … ; 0 0 … 1 0; 0 0 … 1 1])

## Benchmark della funzione `bool2d` ottimizzata

In [21]:
bool2d(assembly)
@btime bool2d($assembly)

# Risultato test: 6.287 ms (44753 allocations: 2.75 MiB)

  6.287 ms (44753 allocations: 2.75 MiB)


([0.0 0.0; 0.0 1.0; … ; 0.921061 0.7894183; 0.5316427 1.7104793], sparse([1, 7, 5, 8, 4, 5, 60, 71, 3, 4  …  58, 59, 69, 70, 63, 74, 64, 73, 70, 74], [1, 1, 2, 2, 3, 3, 3, 3, 4, 4  …  42, 42, 42, 42, 43, 43, 44, 44, 45, 45], Int8[-1, -1, -1, -1, -1, 1, -1, -1, -1, 1  …  1, 1, 1, -1, 1, -1, 1, 1, 1, 1], 74, 45), sparse([11, 1, 18, 2, 8, 3, 9, 4, 10, 11  …  16, 29, 30, 3, 4, 5, 27, 15, 28, 30], [1, 2, 2, 3, 3, 4, 4, 5, 5, 6  …  69, 69, 70, 71, 71, 72, 72, 73, 73, 74], Int8[-1, 1, -1, 1, -1, 1, -1, 1, -1, 1  …  1, -1, 1, -1, 1, -1, 1, -1, 1, -1], 30, 74), Bool[1 1 … 0 0; 1 1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 1])

## Benchmark della funzione `pointInPolygonClassification` originale

In [22]:
V = [0.0 1.1102230246251565e-16 1.0000000000000002 0.9999999876692717; 0.0 -1.0 -1.0 -8.265760442771608e-8]
EV = [[3, 2], [4, 1], [3, 4], [2, 1]]
pnt1 = [0.902095995358517, -0.7029571256300666]
pnt2 = [9.999999999998899e-5, 0.9999]

Lar.pointInPolygonClassification(V, EV)(pnt1)
@btime Lar.pointInPolygonClassification(V, EV)(pnt1)

# Risultato test: 850.524 ns (14 allocations: 1.06 KiB)

  850.524 ns (14 allocations: 1.06 KiB)


"p_in"

## Benchmark della funzione `pointInPolygonClassification` ottimizzata

In [23]:
pointInPolygonClassificationBool(V, EV)(pnt1)
@btime pointInPolygonClassificationBool(V, EV)(pnt1)

# Risultato test: 629.165 ns (10 allocations: 768 bytes)

  629.165 ns (10 allocations: 768 bytes)


"p_in"

## Considerazioni sulle perfomance
Le prestazioni di `bool2d()` sono strettamente influenzate dalla funzione `planar_arrangement()`

In [24]:
V, EV = Lar.struct2lar(assembly)
cop_EW = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
Z = convert(Lar.Points, V')
@btime Lar.Arrangement.planar_arrangement($Z, $cop_EW)

# Risultato test: 5.664 ms (40150 allocations: 2.44 MiB)

  5.664 ms (40150 allocations: 2.44 MiB)


([0.0 0.0; 0.0 1.0; … ; 0.921061 0.7894183; 0.5316427 1.7104793], sparse([1, 7, 5, 8, 4, 5, 60, 71, 3, 4  …  58, 59, 69, 70, 63, 74, 64, 73, 70, 74], [1, 1, 2, 2, 3, 3, 3, 3, 4, 4  …  42, 42, 42, 42, 43, 43, 44, 44, 45, 45], Int8[-1, -1, -1, -1, -1, 1, -1, -1, -1, 1  …  1, 1, 1, -1, 1, -1, 1, 1, 1, 1], 74, 45), sparse([11, 1, 18, 2, 8, 3, 9, 4, 10, 11  …  16, 29, 30, 3, 4, 5, 27, 15, 28, 30], [1, 2, 2, 3, 3, 4, 4, 5, 5, 6  …  69, 69, 70, 71, 71, 72, 72, 73, 73, 74], Int8[-1, 1, -1, 1, -1, 1, -1, 1, -1, 1  …  1, -1, 1, -1, 1, -1, 1, -1, 1, -1], 30, 74))